In [47]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

# Generate a synthetic dataset
X, y = make_classification(n_samples=40, n_features=5, n_informative=3, n_redundant=0, n_clusters_per_class=1, n_classes=3)

# Convert the data to a DataFrame for easier handling
df_features = pd.DataFrame(X, columns=[f'Feature_{i+1}' for i in range(5)])
df_target = pd.DataFrame(y, columns=['Y'])

# Concatenate features and target into one DataFrame
df = pd.concat([df_features, df_target], axis=1)

print(df)


    Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  Y
0    0.821209  -0.249454   1.697793  -1.175892   0.064107  2
1   -2.119163  -0.375135  -1.737217  -1.923423  -0.123764  0
2    0.131719  -1.151343   1.912180  -0.953912   0.132853  2
3   -1.847853   1.828564  -1.739899   0.305378   0.821996  1
4   -1.410634  -1.016693  -0.528842  -1.621271   0.010901  0
5    1.654569  -1.862979   1.088969  -1.594433  -1.249609  2
6   -1.576386  -0.779648  -1.571243   0.570999   0.315930  1
7   -1.394641   0.872065  -1.586451   0.404680   1.675884  1
8   -0.836367  -0.753035   0.692772  -3.515997  -1.324955  2
9   -0.206201   0.731854  -0.506389   1.166799  -0.879734  1
10  -1.090315  -0.440489  -0.858923   1.146621   1.331481  1
11  -0.363141  -0.782933  -0.193642  -0.583964   1.762350  0
12  -0.671268  -0.183820  -1.067632   0.890163   0.417564  1
13  -0.654774  -0.371809  -0.952730  -0.801694  -0.409717  0
14   2.057538  -0.469427   1.393917   0.439180  -0.236643  2
15   0.406995  -1.343292

In [48]:
import pandas as pd
import numpy as np

Y_column = df['Y'].copy()
df.drop('Y', axis=1, inplace=True)

# Identify categorical data (change this based on your actual data)
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

# Standardize only the continuous (non-categorical) columns
continuous_cols = df.columns.difference(categorical_cols)  # Gets the difference, i.e., continuous cols
df[continuous_cols] = (df[continuous_cols] - df[continuous_cols].mean()) / df[continuous_cols].std()

# Filter out outliers in continuous data (|z-score| > 5)
mask = (np.abs(df[continuous_cols]) < 5).all(axis=1)
df = df[mask]

# Reattach the target variable 'Y' to the DataFrame
df['Y'] = Y_column[mask]

In [49]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()

# Apply Label Encoding to each categorical column
for column in categorical_cols:
    # Ensure the column is of type object (string) or category
    if df[column].dtype == 'object' or df[column].dtype.name == 'category':
        df[column] = le.fit_transform(df[column])
        
df['Y'], unique = pd.factorize(df['Y'])

In [50]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Dictionary of models and their reduced hyperparameter grids
models = {
    'Logistic Regression': (LogisticRegression(max_iter=1000), {
        'C': [0.01, 0.1, 1],
        'solver': ['liblinear', 'lbfgs']
    }),
    'KNN': (KNeighborsClassifier(), {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance']
    }),
    'Decision Tree': (DecisionTreeClassifier(), {
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5]
    }),
    'Random Forest': (RandomForestClassifier(), {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10],
        'min_samples_split': [2, 5]
    }),
    'Gradient Boosting': (GradientBoostingClassifier(), {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 4]
    }),
    'XGBoost': (XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'), {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 4]
    }),
    'LightGBM': (LGBMClassifier(), {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1],
        'num_leaves': [31, 50]
    }),
    'CatBoost': (CatBoostClassifier(verbose=0), {
        'iterations': [100, 200, 300],
        'learning_rate': [0.01, 0.1],
        'depth': [4, 6]
    })
}

# Dictionary to store results
results = {}

for name, (model, param_grid) in models.items():
    start_time = time.time()
    
    # Perform GridSearchCV
    grid_search = GridSearchCV(model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)
    
    # Get the best model
    best_model = grid_search.best_estimator_

    # Measure training time for best parameters
    best_param_train_start = time.time()
    best_model.fit(X_train_scaled, y_train)
    best_param_train_time = time.time() - best_param_train_start

    # Measure inference time for best parameters
    inference_start_time = time.time()
    y_pred = best_model.predict(X_test_scaled)
    inference_time = time.time() - inference_start_time
    
    # Calculate total computation time
    computation_time = time.time() - start_time
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    if len(np.unique(y)) == 2:  # Binary classification
        auc = roc_auc_score(y_test, best_model.predict_proba(X_test_scaled)[:, 1])
    else:  # Multiclass classification
        auc = roc_auc_score(y_test, best_model.predict_proba(X_test_scaled), multi_class='ovr', average='macro')

    results[name] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'Training Time (Best Params)': best_param_train_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': grid_search.best_params_
    }

# Convert results to a DataFrame
result = pd.DataFrame(results).T

print(result)

c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [07:45:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 32, number of used features: 0
[LightGBM] [Info] Start training from score -1.067841
[LightGBM] [Info] Start training from score -1.268511
[LightGBM] [Info] Start training from score -0.980829
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the

In [51]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.neural_network import MLPClassifier

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the MLP model and its hyperparameter grid
mlp = MLPClassifier(max_iter=1000, random_state=42)
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive']
}

# Perform GridSearchCV with StratifiedKFold
start_time = time.time()
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
grid_search = GridSearchCV(mlp, param_grid=param_grid, cv=skf, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Measure training time for best parameters
best_param_train_start = time.time()
best_model.fit(X_train_scaled, y_train)
training_time = time.time() - best_param_train_start

# Measure inference time for best parameters
inference_start_time = time.time()
y_pred = best_model.predict(X_test_scaled)
inference_time = time.time() - inference_start_time

# Calculate total computation time
computation_time = time.time() - start_time

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
if len(np.unique(y)) == 2:  # Binary classification
    auc = roc_auc_score(y_test, best_model.predict_proba(X_test_scaled)[:, 1])
else:  # Multiclass classification
    auc = roc_auc_score(y_test, best_model.predict_proba(X_test_scaled), multi_class='ovr', average='macro')

# Store results in the existing result DataFrame
result.loc['MLP'] = [accuracy, auc, training_time, inference_time, computation_time, grid_search.best_params_]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in grid_search.best_params_.items():
    print(f"{param}: {value}")

                    Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression     0.75  0.927778                    0.002991   
KNN                     0.75       1.0                    0.000996   
Decision Tree          0.875  0.931746                       0.001   
Random Forest           0.75  0.933333                    0.118717   
Gradient Boosting      0.875  0.920635                    0.247005   
XGBoost                 0.75  0.805556                    0.082778   
LightGBM               0.125       0.5                    0.010972   
CatBoost                0.75  0.977778                    0.119442   
MLP                     0.75       1.0                    0.513627   

                    Inference Time (Best Params) Computation Time (Total)  \
Logistic Regression                     0.002992                 5.321193   
KNN                                     0.002262                 0.203027   
Decision Tree                                0.0                 0.0

In [52]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

# Define the DNN model
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(DNN, self).__init__()
        layers = []
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.ReLU())
            input_dim = hidden_dim
        layers.append(nn.Linear(input_dim, output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# Define the objective function for Optuna
def objective(trial):
    # Define hyperparameters to tune
    hidden_dims = [trial.suggest_int(f'hidden_dim_{i}', 32, 256) for i in range(3)]
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the model
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    model = DNN(input_dim, hidden_dims, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Start timing the entire process
start_time = time.time()

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final model with the best hyperparameters
input_dim = X_train.shape[1]
output_dim = len(np.unique(y))
best_model = DNN(input_dim, [best_params[f'hidden_dim_{i}'] for i in range(3)], output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['DNN'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 07:46:16,958] A new study created in memory with name: no-name-7560ba0e-7c0e-41f7-a212-760c6068857a


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\1406562829.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:46:17,319] Trial 0 finished with value: 0.875 and parameters: {'hidden_dim_0': 188, 'hidden_dim_1': 168, 'hidden_dim_2': 143, 'learning_rate': 0.0025503930381602113, 'batch_size': 64, 'num_epochs': 65}. Best is trial 0 with value: 0.875.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\1406562829.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:46:17,609] Trial 1 finished with value: 

                    Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression     0.75  0.927778                    0.002991   
KNN                     0.75       1.0                    0.000996   
Decision Tree          0.875  0.931746                       0.001   
Random Forest           0.75  0.933333                    0.118717   
Gradient Boosting      0.875  0.920635                    0.247005   
XGBoost                 0.75  0.805556                    0.082778   
LightGBM               0.125       0.5                    0.010972   
CatBoost                0.75  0.977778                    0.119442   
MLP                     0.75       1.0                    0.513627   
DNN                     0.75  0.922222                     0.23946   

                    Inference Time (Best Params) Computation Time (Total)  \
Logistic Regression                     0.002992                 5.321193   
KNN                                     0.002262                 0.203027  

In [53]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class CrossLayer(nn.Module):
    def __init__(self, input_dim):
        super(CrossLayer, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(input_dim, 1))
        self.bias = nn.Parameter(torch.Tensor(input_dim, 1))
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)

    def forward(self, x0, x):
        x = x.unsqueeze(2)
        x0 = x0.unsqueeze(2)
        interaction = torch.matmul(x0, torch.matmul(x.transpose(1, 2), self.weight))
        return x0.squeeze(2) + interaction.squeeze(2) + self.bias.T

class DCN(nn.Module):
    def __init__(self, input_dim, cross_layers, hidden_layers, output_dim):
        super(DCN, self).__init__()
        self.cross_layers = nn.ModuleList([CrossLayer(input_dim) for _ in range(cross_layers)])
        
        deep_layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                deep_layers.append(nn.Linear(input_dim, hidden_layers[i]))
            else:
                deep_layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            deep_layers.append(nn.ReLU())
        self.deep_net = nn.Sequential(*deep_layers)
        
        self.final_layer = nn.Linear(input_dim + hidden_layers[-1], output_dim)

    def forward(self, x):
        cross_out = x
        for layer in self.cross_layers:
            cross_out = layer(x, cross_out)
        deep_out = self.deep_net(x)
        concat_out = torch.cat([cross_out, deep_out], dim=1)
        return self.final_layer(concat_out)

def objective(trial):
    # Define hyperparameters to tune
    cross_layers = trial.suggest_int('cross_layers', 1, 5)
    hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the model
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    model = DCN(input_dim, cross_layers, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Start timing the entire process
start_time = time.time()

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final model with the best hyperparameters
input_dim = X_train.shape[1]
output_dim = len(np.unique(y))
best_model = DCN(input_dim, best_params['cross_layers'], 
                 [best_params[f'hidden_layer_{i}'] for i in range(3)], 
                 output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['DCN'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 07:46:39,311] A new study created in memory with name: no-name-3900da39-de46-45da-b04d-8140173eabe5


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\1122408048.py:77: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:46:39,632] Trial 0 finished with value: 0.875 and parameters: {'cross_layers': 4, 'hidden_layer_0': 184, 'hidden_layer_1': 169, 'hidden_layer_2': 100, 'learning_rate': 0.0003024897105741653, 'batch_size': 128, 'num_epochs': 36}. Best is trial 0 with value: 0.875.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\1122408048.py:77: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:46:39,943] Tri

                    Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression     0.75  0.927778                    0.002991   
KNN                     0.75       1.0                    0.000996   
Decision Tree          0.875  0.931746                       0.001   
Random Forest           0.75  0.933333                    0.118717   
Gradient Boosting      0.875  0.920635                    0.247005   
XGBoost                 0.75  0.805556                    0.082778   
LightGBM               0.125       0.5                    0.010972   
CatBoost                0.75  0.977778                    0.119442   
MLP                     0.75       1.0                    0.513627   
DNN                     0.75  0.922222                     0.23946   
DCN                    0.875  0.927778                    0.297203   

                    Inference Time (Best Params) Computation Time (Total)  \
Logistic Regression                     0.002992                 5.321193   
KNN  

In [54]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class WideAndDeepNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(WideAndDeepNetwork, self).__init__()
        
        # Wide part
        self.wide = nn.Linear(input_dim, output_dim)
        
        # Deep part
        deep_layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                deep_layers.append(nn.Linear(input_dim, hidden_layers[i]))
            else:
                deep_layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            deep_layers.append(nn.ReLU())
        deep_layers.append(nn.Linear(hidden_layers[-1], output_dim))
        self.deep = nn.Sequential(*deep_layers)

    def forward(self, x):
        wide_out = self.wide(x)
        deep_out = self.deep(x)
        return wide_out + deep_out

def objective(trial):
    # Define hyperparameters to tune
    hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the model
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    model = WideAndDeepNetwork(input_dim, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Start timing the entire process
start_time = time.time()

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final model with the best hyperparameters
input_dim = X_train.shape[1]
output_dim = len(np.unique(y))
best_model = WideAndDeepNetwork(input_dim, 
                                [best_params[f'hidden_layer_{i}'] for i in range(3)], 
                                output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['Wide_and_Deep'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 07:47:03,528] A new study created in memory with name: no-name-e4818d4b-c71f-436d-bd5b-0e49d9597479


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\3180404897.py:61: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:47:03,937] Trial 0 finished with value: 0.875 and parameters: {'hidden_layer_0': 56, 'hidden_layer_1': 227, 'hidden_layer_2': 118, 'learning_rate': 0.0041430798801775125, 'batch_size': 128, 'num_epochs': 84}. Best is trial 0 with value: 0.875.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\3180404897.py:61: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:47:03,997] Trial 1 finished with v

                    Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression     0.75  0.927778                    0.002991   
KNN                     0.75       1.0                    0.000996   
Decision Tree          0.875  0.931746                       0.001   
Random Forest           0.75  0.933333                    0.118717   
Gradient Boosting      0.875  0.920635                    0.247005   
XGBoost                 0.75  0.805556                    0.082778   
LightGBM               0.125       0.5                    0.010972   
CatBoost                0.75  0.977778                    0.119442   
MLP                     0.75       1.0                    0.513627   
DNN                     0.75  0.922222                     0.23946   
DCN                    0.875  0.927778                    0.297203   
Wide_and_Deep          0.875  0.830556                    0.449732   

                    Inference Time (Best Params) Computation Time (Total)  \
Logistic Reg

In [55]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from xgboost import XGBClassifier
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                layers.append(nn.Linear(input_dim, hidden_layers[i]))
            else:
                layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_layers[-1], output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune for XGBoost
    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-4, 1e-1),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }

    # Train XGBoost model
    xgb_model = XGBClassifier(**xgb_params, use_label_encoder=False, eval_metric='mlogloss')
    xgb_model.fit(X_train_scaled, y_train)
    
    # Extract features using XGBoost
    X_train_transformed = xgb_model.apply(X_train_scaled)
    X_test_transformed = xgb_model.apply(X_test_scaled)
    
    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
    y_train_tensor = torch.LongTensor(y_train.values).to(device)
    y_test_tensor = torch.LongTensor(y_test.values).to(device)
    
    # Define hyperparameters to tune for Neural Network
    hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
    nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the Neural Network model
    input_dim = X_train_transformed.shape[1]
    output_dim = len(np.unique(y))
    model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_transformed_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final XGBoost model with the best hyperparameters
xgb_best_params = {
    'n_estimators': best_params['n_estimators'],
    'max_depth': best_params['max_depth'],
    'learning_rate': best_params['xgb_learning_rate'],
    'subsample': best_params['subsample'],
    'colsample_bytree': best_params['colsample_bytree']
}
xgb_model = XGBClassifier(**xgb_best_params, use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train_scaled, y_train)

# Extract features using XGBoost
X_train_transformed = xgb_model.apply(X_train_scaled)
X_test_transformed = xgb_model.apply(X_test_scaled)

# Convert to PyTorch tensors
X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

# Train the final Neural Network model with the best hyperparameters
nn_best_params = {
    'hidden_layers': [best_params[f'hidden_layer_{i}'] for i in range(3)],
    'learning_rate': best_params['nn_learning_rate'],
    'batch_size': best_params['batch_size'],
    'num_epochs': best_params['num_epochs']
}
input_dim = X_train_transformed.shape[1]
output_dim = len(np.unique(y))
best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(nn_best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_transformed_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['XGBoost + NN'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 07:47:26,601] A new study created in memory with name: no-name-14cde900-b97d-4cbb-adbc-5872cc99c59a


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\13981151.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-4, 1e-1),
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [07:47:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\13981151.py:75: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  nn_learning_rate = trial.suggest_loguniform('nn_learning_rate

                    Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression     0.75  0.927778                    0.002991   
KNN                     0.75       1.0                    0.000996   
Decision Tree          0.875  0.931746                       0.001   
Random Forest           0.75  0.933333                    0.118717   
Gradient Boosting      0.875  0.920635                    0.247005   
XGBoost                 0.75  0.805556                    0.082778   
LightGBM               0.125       0.5                    0.010972   
CatBoost                0.75  0.977778                    0.119442   
MLP                     0.75       1.0                    0.513627   
DNN                     0.75  0.922222                     0.23946   
DCN                    0.875  0.927778                    0.297203   
Wide_and_Deep          0.875  0.830556                    0.449732   
XGBoost + NN           0.875  0.888889                    0.364206   

                   

In [56]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from lightgbm import LGBMClassifier
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                layers.append(nn.Linear(input_dim, hidden_layers[i]))
            else:
                layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_layers[-1], output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune for LightGBM
    lgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('lgb_learning_rate', 1e-4, 1e-1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }

    # Train LightGBM model
    lgb_model = LGBMClassifier(**lgb_params)
    lgb_model.fit(X_train_scaled, y_train)
    
    # Extract features using LightGBM
    X_train_transformed = lgb_model.predict_proba(X_train_scaled)
    X_test_transformed = lgb_model.predict_proba(X_test_scaled)
    
    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
    y_train_tensor = torch.LongTensor(y_train.values).to(device)
    y_test_tensor = torch.LongTensor(y_test.values).to(device)
    
    # Define hyperparameters to tune for Neural Network
    hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
    nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the Neural Network model
    input_dim = X_train_transformed.shape[1]
    output_dim = len(np.unique(y))
    model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_transformed_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final LightGBM model with the best hyperparameters
lgb_best_params = {
    'n_estimators': best_params['n_estimators'],
    'max_depth': best_params['max_depth'],
    'learning_rate': best_params['lgb_learning_rate'],
    'num_leaves': best_params['num_leaves'],
    'subsample': best_params['subsample'],
    'colsample_bytree': best_params['colsample_bytree']
}
lgb_model = LGBMClassifier(**lgb_best_params)
lgb_model.fit(X_train_scaled, y_train)

# Extract features using LightGBM
X_train_transformed = lgb_model.predict_proba(X_train_scaled)
X_test_transformed = lgb_model.predict_proba(X_test_scaled)

# Convert to PyTorch tensors
X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

# Train the final Neural Network model with the best hyperparameters
nn_best_params = {
    'hidden_layers': [best_params[f'hidden_layer_{i}'] for i in range(3)],
    'learning_rate': best_params['nn_learning_rate'],
    'batch_size': best_params['batch_size'],
    'num_epochs': best_params['num_epochs']
}
input_dim = X_train_transformed.shape[1]
output_dim = len(np.unique(y))
best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(nn_best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_transformed_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['LightGBM + NN'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 07:48:08,387] A new study created in memory with name: no-name-643bb24c-82bd-4848-b6be-2c32307b6582


Using device: cpu
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 32, number of used features: 0
[LightGBM] [Info] Start training from score -1.163151
[LightGBM] [Info] Start training from score -1.067841
[LightGBM] [Info] Start training from score -1.067841
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more le

C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\1256629833.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('lgb_learning_rate', 1e-4, 1e-1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\1256629833.py:76: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:48:08,639] Trial 0 finished with value: 0.375 and parameters: {'n_estimators': 135, 'max_depth': 6, 'lgb_learning_rate': 0.004344106798222012, 'num_leaves': 87, 'subsample': 0.627133286041994, 'colsample_bytree': 0.678357634226493, 'hidden_layer_0': 102, 'hidden_layer_1': 177, 'hidd

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 32, number of used features: 0
[LightGBM] [Info] Start training from score -1.163151
[LightGBM] [Info] Start training from score -1.067841
[LightGBM] [Info] Start training from score -1.067841
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the

[I 2024-07-26 07:48:09,011] Trial 2 finished with value: 0.25 and parameters: {'n_estimators': 293, 'max_depth': 6, 'lgb_learning_rate': 0.0177700758435059, 'num_leaves': 31, 'subsample': 0.7018695501799963, 'colsample_bytree': 0.6981637430902841, 'hidden_layer_0': 152, 'hidden_layer_1': 244, 'hidden_layer_2': 233, 'nn_learning_rate': 0.007261364537486744, 'batch_size': 64, 'num_epochs': 51}. Best is trial 0 with value: 0.375.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\1256629833.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('lgb_learning_rate', 1e-4, 1e-1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\1256629833.py:76: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/t

...

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 32, number of used features: 0
[LightGBM] [Info] Start training from score -1.163151
[LightGBM] [Info] Start training from score -1.067841
[LightGBM] [Info] Start training from score -1.067841
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the

[I 2024-07-26 07:48:26,683] Trial 47 finished with value: 0.25 and parameters: {'n_estimators': 179, 'max_depth': 9, 'lgb_learning_rate': 0.0016778100197626022, 'num_leaves': 54, 'subsample': 0.6945726339776863, 'colsample_bytree': 0.7340095710074102, 'hidden_layer_0': 146, 'hidden_layer_1': 192, 'hidden_layer_2': 164, 'nn_learning_rate': 0.022579975826739977, 'batch_size': 32, 'num_epochs': 45}. Best is trial 0 with value: 0.375.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\1256629833.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('lgb_learning_rate', 1e-4, 1e-1),


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 32, number of used features: 0
[LightGBM] [Info] Start training from score -1.163151
[LightGBM] [Info] Start training from score -1.067841
[LightGBM] [Info] Start training from score -1.067841
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the

C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\1256629833.py:76: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:48:27,634] Trial 48 finished with value: 0.375 and parameters: {'n_estimators': 51, 'max_depth': 3, 'lgb_learning_rate': 0.04350613731361228, 'num_leaves': 87, 'subsample': 0.647892252543757, 'colsample_bytree': 0.6212991141149389, 'hidden_layer_0': 68, 'hidden_layer_1': 233, 'hidden_layer_2': 185, 'nn_learning_rate': 0.007821446893751976, 'batch_size': 256, 'num_epochs': 94}. Best is trial 0 with value: 0.375.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\1256629833.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 32, number of used features: 0
[LightGBM] [Info] Start training from score -1.163151
[LightGBM] [Info] Start training from score -1.067841
[LightGBM] [Info] Start training from score -1.067841
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the

In [57]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from catboost import CatBoostClassifier
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                layers.append(nn.Linear(input_dim, hidden_layers[i]))
            else:
                layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_layers[-1], output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune for CatBoost
    catboost_params = {
        'iterations': trial.suggest_int('iterations', 50, 300),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('catboost_learning_rate', 1e-4, 1e-1),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-4, 1e-1),
        'border_count': trial.suggest_int('border_count', 32, 255)
    }

    # Train CatBoost model
    catboost_model = CatBoostClassifier(**catboost_params, verbose=0)
    catboost_model.fit(X_train_scaled, y_train)
    
    # Extract features using CatBoost
    X_train_transformed = catboost_model.predict_proba(X_train_scaled)
    X_test_transformed = catboost_model.predict_proba(X_test_scaled)
    
    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
    y_train_tensor = torch.LongTensor(y_train.values).to(device)
    y_test_tensor = torch.LongTensor(y_test.values).to(device)
    
    # Define hyperparameters to tune for Neural Network
    hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
    nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the Neural Network model
    input_dim = X_train_transformed.shape[1]
    output_dim = len(np.unique(y))
    model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_transformed_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final CatBoost model with the best hyperparameters
catboost_best_params = {
    'iterations': best_params['iterations'],
    'depth': best_params['depth'],
    'learning_rate': best_params['catboost_learning_rate'],
    'l2_leaf_reg': best_params['l2_leaf_reg'],
    'border_count': best_params['border_count']
}
catboost_model = CatBoostClassifier(**catboost_best_params, verbose=0)
catboost_model.fit(X_train_scaled, y_train)

# Extract features using CatBoost
X_train_transformed = catboost_model.predict_proba(X_train_scaled)
X_test_transformed = catboost_model.predict_proba(X_test_scaled)

# Convert to PyTorch tensors
X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

# Train the final Neural Network model with the best hyperparameters
nn_best_params = {
    'hidden_layers': [best_params[f'hidden_layer_{i}'] for i in range(3)],
    'learning_rate': best_params['nn_learning_rate'],
    'batch_size': best_params['batch_size'],
    'num_epochs': best_params['num_epochs']
}
input_dim = X_train_transformed.shape[1]
output_dim = len(np.unique(y))
best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(nn_best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_transformed_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['CatBoost + NN'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 07:48:28,289] A new study created in memory with name: no-name-c0bbadbb-c617-46f0-8765-0f91504a61c6


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\3359856436.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('catboost_learning_rate', 1e-4, 1e-1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\3359856436.py:55: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-4, 1e-1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\3359856436.py:75: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  nn_learning_rate = trial.sugg

                    Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression     0.75  0.927778                    0.002991   
KNN                     0.75       1.0                    0.000996   
Decision Tree          0.875  0.931746                       0.001   
Random Forest           0.75  0.933333                    0.118717   
Gradient Boosting      0.875  0.920635                    0.247005   
XGBoost                 0.75  0.805556                    0.082778   
LightGBM               0.125       0.5                    0.010972   
CatBoost                0.75  0.977778                    0.119442   
MLP                     0.75       1.0                    0.513627   
DNN                     0.75  0.922222                     0.23946   
DCN                    0.875  0.927778                    0.297203   
Wide_and_Deep          0.875  0.830556                    0.449732   
XGBoost + NN           0.875  0.888889                    0.364206   
LightGBM + NN       

In [58]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class AutoInt(nn.Module):
    def __init__(self, input_dim, embedding_dim, num_heads, num_layers):
        super(AutoInt, self).__init__()
        self.embedding = nn.Linear(input_dim, embedding_dim)
        self.attention_layers = nn.ModuleList([
            nn.MultiheadAttention(embedding_dim, num_heads) for _ in range(num_layers)
        ])
        self.fc = nn.Linear(embedding_dim, 1)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        for attn_layer in self.attention_layers:
            x, _ = attn_layer(x, x, x)
        x = x.squeeze(1)
        x = self.fc(x)
        return x

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                layers.append(nn.Linear(input_dim, hidden_layers[i]))
            else:
                layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_layers[-1], output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune for AutoInt
    num_heads = trial.suggest_int('num_heads', 1, 8)
    embedding_dim = trial.suggest_int('embedding_dim', num_heads, 64, step=num_heads)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    
    # Train AutoInt model
    autoint_model = AutoInt(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
    optimizer = optim.Adam(autoint_model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor.float().unsqueeze(1))
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    for epoch in range(10):  # Fixed number of epochs for AutoInt
        autoint_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = autoint_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    # Extract features using AutoInt
    autoint_model.eval()
    with torch.no_grad():
        X_train_transformed = autoint_model.embedding(X_train_tensor).cpu().numpy()
        X_test_transformed = autoint_model.embedding(X_test_tensor).cpu().numpy()
    
    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
    
    # Define hyperparameters to tune for Neural Network
    hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
    nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the Neural Network model
    input_dim = X_train_transformed.shape[1]
    output_dim = len(np.unique(y))
    model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_transformed_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final AutoInt model with the best hyperparameters
embedding_dim = best_params['embedding_dim']
num_heads = best_params['num_heads']
num_layers = best_params['num_layers']
autoint_model = AutoInt(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
optimizer = optim.Adam(autoint_model.parameters(), lr=0.001)
criterion = nn.MSELoss()

train_dataset = TensorDataset(X_train_tensor, y_train_tensor.float().unsqueeze(1))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

for epoch in range(10):  # Fixed number of epochs for AutoInt
    autoint_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = autoint_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

# Extract features using AutoInt
autoint_model.eval()
with torch.no_grad():
    X_train_transformed = autoint_model.embedding(X_train_tensor).cpu().numpy()
    X_test_transformed = autoint_model.embedding(X_test_tensor).cpu().numpy()

# Convert to PyTorch tensors
X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)

# Train the final Neural Network model with the best hyperparameters
nn_best_params = {
    'hidden_layers': [best_params[f'hidden_layer_{i}'] for i in range(3)],
    'learning_rate': best_params['nn_learning_rate'],
    'batch_size': best_params['batch_size'],
    'num_epochs': best_params['num_epochs']
}
input_dim = X_train_transformed.shape[1]
output_dim = len(np.unique(y))
best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(nn_best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_transformed_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['AutoInt'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 07:49:17,041] A new study created in memory with name: no-name-f2c415c6-28ee-431f-8031-586e30dec0b2


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\2123632112.py:106: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:49:17,308] Trial 0 finished with value: 0.625 and parameters: {'num_heads': 1, 'embedding_dim': 60, 'num_layers': 3, 'hidden_layer_0': 59, 'hidden_layer_1': 248, 'hidden_layer_2': 107, 'nn_learning_rate': 0.00014720027560375466, 'batch_size': 32, 'num_epochs': 17}. Best is trial 0 with value: 0.625.
c:\Users\Tommy\anaconda3\Lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [3, 64] and step=3, but the range is not divisible by `step`. It will be replaced by [3, 63].
  warnings.warn(
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\2123632112.py:106: FutureWarning: suggest_loguniform 

                    Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression     0.75  0.927778                    0.002991   
KNN                     0.75       1.0                    0.000996   
Decision Tree          0.875  0.931746                       0.001   
Random Forest           0.75  0.933333                    0.118717   
Gradient Boosting      0.875  0.920635                    0.247005   
XGBoost                 0.75  0.805556                    0.082778   
LightGBM               0.125       0.5                    0.010972   
CatBoost                0.75  0.977778                    0.119442   
MLP                     0.75       1.0                    0.513627   
DNN                     0.75  0.922222                     0.23946   
DCN                    0.875  0.927778                    0.297203   
Wide_and_Deep          0.875  0.830556                    0.449732   
XGBoost + NN           0.875  0.888889                    0.364206   
LightGBM + NN       

In [59]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class FTTransformer(nn.Module):
    def __init__(self, input_dim, embedding_dim, num_heads, num_layers):
        super(FTTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, embedding_dim)
        self.transformer_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads) for _ in range(num_layers)
        ])
        self.fc = nn.Linear(embedding_dim, input_dim)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        for transformer_layer in self.transformer_layers:
            x = transformer_layer(x)
        x = x.squeeze(1)
        x = self.fc(x)
        return x

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                layers.append(nn.Linear(input_dim, hidden_layers[i]))
            else:
                layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_layers[-1], output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune for FT-Transformer
    num_heads = trial.suggest_int('num_heads', 1, 8)
    embedding_dim = trial.suggest_int('embedding_dim', num_heads, 64, step=num_heads)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    
    # Train FT-Transformer model
    ft_transformer_model = FTTransformer(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
    optimizer = optim.Adam(ft_transformer_model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor.float().unsqueeze(1))
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    for epoch in range(10):  # Fixed number of epochs for FT-Transformer
        ft_transformer_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = ft_transformer_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    # Extract features using FT-Transformer
    ft_transformer_model.eval()
    with torch.no_grad():
        X_train_transformed = ft_transformer_model.embedding(X_train_tensor).cpu().numpy()
        X_test_transformed = ft_transformer_model.embedding(X_test_tensor).cpu().numpy()
    
    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
    
    # Define hyperparameters to tune for Neural Network
    hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
    nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the Neural Network model
    input_dim = X_train_transformed.shape[1]
    output_dim = len(np.unique(y))
    model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_transformed_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final FT-Transformer model with the best hyperparameters
embedding_dim = best_params['embedding_dim']
num_heads = best_params['num_heads']
num_layers = best_params['num_layers']
ft_transformer_model = FTTransformer(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
optimizer = optim.Adam(ft_transformer_model.parameters(), lr=0.001)
criterion = nn.MSELoss()

train_dataset = TensorDataset(X_train_tensor, y_train_tensor.float().unsqueeze(1))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

for epoch in range(10):  # Fixed number of epochs for FT-Transformer
    ft_transformer_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = ft_transformer_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

# Extract features using FT-Transformer
ft_transformer_model.eval()
with torch.no_grad():
    X_train_transformed = ft_transformer_model.embedding(X_train_tensor).cpu().numpy()
    X_test_transformed = ft_transformer_model.embedding(X_test_tensor).cpu().numpy()

# Convert to PyTorch tensors
X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)

# Train the final Neural Network model with the best hyperparameters
nn_best_params = {
    'hidden_layers': [best_params[f'hidden_layer_{i}'] for i in range(3)],
    'learning_rate': best_params['nn_learning_rate'],
    'batch_size': best_params['batch_size'],
    'num_epochs': best_params['num_epochs']
}
input_dim = X_train_transformed.shape[1]
output_dim = len(np.unique(y))
best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(nn_best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_transformed_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['FT-Transformer'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 07:49:51,798] A new study created in memory with name: no-name-14fa1cd7-5559-4abd-8519-080704ddae98


Using device: cpu


c:\Users\Tommy\anaconda3\Lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [5, 64] and step=5, but the range is not divisible by `step`. It will be replaced by [5, 60].
  warnings.warn(
c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\299350481.py:106: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:50:00,084] Trial 0 finished with value: 0.875 and par

                    Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression     0.75  0.927778                    0.002991   
KNN                     0.75       1.0                    0.000996   
Decision Tree          0.875  0.931746                       0.001   
Random Forest           0.75  0.933333                    0.118717   
Gradient Boosting      0.875  0.920635                    0.247005   
XGBoost                 0.75  0.805556                    0.082778   
LightGBM               0.125       0.5                    0.010972   
CatBoost                0.75  0.977778                    0.119442   
MLP                     0.75       1.0                    0.513627   
DNN                     0.75  0.922222                     0.23946   
DCN                    0.875  0.927778                    0.297203   
Wide_and_Deep          0.875  0.830556                    0.449732   
XGBoost + NN           0.875  0.888889                    0.364206   
LightGBM + NN       

In [60]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                layers.append(nn.Linear(input_dim, hidden_layers[i]))
            else:
                layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_layers[-1], output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune for Neural Network
    num_layers = trial.suggest_int('num_layers', 1, 5)
    hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(num_layers)]
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the Neural Network model
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final Neural Network model with the best hyperparameters
input_dim = X_train.shape[1]
output_dim = len(np.unique(y))
best_model = NeuralNetwork(input_dim, 
                           [best_params[f'hidden_layer_{i}'] for i in range(best_params['num_layers'])], 
                           output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['Neural Architecture Search'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 07:50:36,225] A new study created in memory with name: no-name-ff75fcac-0335-43eb-b5f4-33102e0bea4b


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\3796604553.py:58: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:50:37,197] Trial 0 finished with value: 0.75 and parameters: {'num_layers': 1, 'hidden_layer_0': 82, 'learning_rate': 0.009076696335475115, 'batch_size': 64, 'num_epochs': 59}. Best is trial 0 with value: 0.75.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\3796604553.py:58: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:50:37,425] Trial 1 finished with value: 0.625 and parameters: {'num

                           Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression            0.75  0.927778                    0.002991   
KNN                            0.75       1.0                    0.000996   
Decision Tree                 0.875  0.931746                       0.001   
Random Forest                  0.75  0.933333                    0.118717   
Gradient Boosting             0.875  0.920635                    0.247005   
XGBoost                        0.75  0.805556                    0.082778   
LightGBM                      0.125       0.5                    0.010972   
CatBoost                       0.75  0.977778                    0.119442   
MLP                            0.75       1.0                    0.513627   
DNN                            0.75  0.922222                     0.23946   
DCN                           0.875  0.927778                    0.297203   
Wide_and_Deep                 0.875  0.830556                    0.449732   

In [61]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class NODE(nn.Module):
    def __init__(self, input_dim, num_layers, num_trees, tree_dim, output_dim):
        super(NODE, self).__init__()
        self.layers = nn.ModuleList()
        for _ in range(num_layers):
            layer = nn.ModuleList()
            for _ in range(num_trees):
                tree = nn.Sequential(
                    nn.Linear(input_dim, tree_dim),
                    nn.ReLU(),
                    nn.Linear(tree_dim, 1)
                )
                layer.append(tree)
            self.layers.append(layer)
        self.output = nn.Linear(num_layers * num_trees, output_dim)

    def forward(self, x):
        tree_outputs = []
        for layer in self.layers:
            layer_outputs = []
            for tree in layer:
                layer_outputs.append(tree(x))
            layer_output = torch.cat(layer_outputs, dim=1)
            tree_outputs.append(layer_output)
        x = torch.cat(tree_outputs, dim=1)
        return self.output(x)

def objective(trial):
    # Define hyperparameters to tune for NODE
    num_layers = trial.suggest_int('num_layers', 1, 5)
    num_trees = trial.suggest_int('num_trees', 1, 10)
    tree_dim = trial.suggest_int('tree_dim', 8, 64)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the NODE model
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    model = NODE(input_dim, num_layers, num_trees, tree_dim, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final NODE model with the best hyperparameters
input_dim = X_train.shape[1]
output_dim = len(np.unique(y))
best_model = NODE(input_dim, 
                  best_params['num_layers'], 
                  best_params['num_trees'], 
                  best_params['tree_dim'], 
                  output_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_tensor)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    
    _, predicted = torch.max(outputs, 1)
    y_pred = predicted.cpu().numpy()
    proba = torch.softmax(outputs, dim=1).cpu().numpy()

    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        auc = roc_auc_score(y_true, proba[:, 1])
    else:  # Multi-class classification
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['NODE'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 07:50:53,824] A new study created in memory with name: no-name-cb598485-e337-47ff-a458-342b40d7bac2


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\4111450121.py:70: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:50:54,275] Trial 0 finished with value: 0.75 and parameters: {'num_layers': 4, 'num_trees': 5, 'tree_dim': 59, 'learning_rate': 0.006039884739845829, 'batch_size': 32, 'num_epochs': 21}. Best is trial 0 with value: 0.75.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\4111450121.py:70: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:50:56,201] Trial 1 finished with value: 0.75 and paramete

                           Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression            0.75  0.927778                    0.002991   
KNN                            0.75       1.0                    0.000996   
Decision Tree                 0.875  0.931746                       0.001   
Random Forest                  0.75  0.933333                    0.118717   
Gradient Boosting             0.875  0.920635                    0.247005   
XGBoost                        0.75  0.805556                    0.082778   
LightGBM                      0.125       0.5                    0.010972   
CatBoost                       0.75  0.977778                    0.119442   
MLP                            0.75       1.0                    0.513627   
DNN                            0.75  0.922222                     0.23946   
DCN                           0.875  0.927778                    0.297203   
Wide_and_Deep                 0.875  0.830556                    0.449732   

In [62]:
import pandas as pd
import numpy as np
import time
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

def objective(trial):
    # Define hyperparameters to tune for TabNet
    n_d = trial.suggest_int('n_d', 8, 64)
    n_a = trial.suggest_int('n_a', 8, 64)
    n_steps = trial.suggest_int('n_steps', 3, 10)
    gamma = trial.suggest_float('gamma', 1.0, 2.0)
    lambda_sparse = trial.suggest_loguniform('lambda_sparse', 1e-6, 1e-3)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the TabNet model
    model = TabNetClassifier(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        lambda_sparse=lambda_sparse,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=learning_rate),
        device_name=device
    )

    # Training
    model.fit(
        X_train=X_train_scaled, y_train=y_train.values,
        eval_set=[(X_test_scaled, y_test.values)],
        eval_name=['val'],
        eval_metric=['accuracy'],
        max_epochs=num_epochs,
        patience=10,
        batch_size=batch_size,
        virtual_batch_size=batch_size // 2,
        num_workers=0,
        drop_last=False
    )

    # Evaluation
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final TabNet model with the best hyperparameters
best_model = TabNetClassifier(
    n_d=best_params['n_d'],
    n_a=best_params['n_a'],
    n_steps=best_params['n_steps'],
    gamma=best_params['gamma'],
    lambda_sparse=best_params['lambda_sparse'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=best_params['learning_rate']),
    device_name=device
)

training_start_time = time.time()
best_model.fit(
    X_train=X_train_scaled, y_train=y_train.values,
    eval_set=[(X_test_scaled, y_test.values)],
    eval_name=['val'],
    eval_metric=['accuracy'],
    max_epochs=best_params['num_epochs'],
    patience=10,
    batch_size=best_params['batch_size'],
    virtual_batch_size=best_params['batch_size'] // 2,
    num_workers=0,
    drop_last=False
)
training_time = time.time() - training_start_time

# Evaluation
y_pred = best_model.predict(X_test_scaled)
inference_start_time = time.time()
y_pred_proba = best_model.predict_proba(X_test_scaled)
inference_time = time.time() - inference_start_time

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
if len(np.unique(y)) == 2:  # Binary classification
    auc = roc_auc_score(y_test, y_pred_proba[:, 1])
else:  # Multiclass classification
    auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['TabNet'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 07:51:28,267] A new study created in memory with name: no-name-65d91aa5-2e31-4c38-9900-2149d78c7c9e


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\716812650.py:43: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lambda_sparse = trial.suggest_loguniform('lambda_sparse', 1e-6, 1e-3)
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\716812650.py:44: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 3.03484 | val_accuracy: 0.375   |  0:00:00s
epoch 1  | loss: 2.53832 | val_accuracy: 0.25    |  0:00:00s
epoch 2  | loss: 2.39976 | val_accuracy: 0.375   |  0:00:00s
epoch 3  | loss: 2.10387 | val_accuracy: 0.25    |  0:00:00s
epoch 4  | loss: 1.88961 | val_accuracy: 0.375   |  0:00:00s
epoch 5  | loss: 1.67937 | val_accuracy: 0.375   |  0:00:00s
epoch 6  | loss: 1.53497 | val_accuracy: 0.375   |  0:00:00s
epoch 7  | loss: 1.47155 | val_accuracy: 0.375   |  0:00:00s
epoch 8  | loss: 1.25748 | val_accuracy: 0.5     |  0:00:00s
epoch 9  | loss: 1.15941 | val_accuracy: 0.375   |  0:00:00s
epoch 10 | loss: 1.01234 | val_accuracy: 0.375   |  0:00:00s
epoch 11 | loss: 0.85431 | val_accuracy: 0.375   |  0:00:00s
epoch 12 | loss: 0.7321  | val_accuracy: 0.375   |  0:00:00s
epoch 13 | loss: 0.63131 | val_accuracy: 0.25    |  0:00:01s
epoch 14 | loss: 0.53783 | val_accuracy: 0.25    |  0:00:01s
epoch 15 | loss: 0.45452 | val_accuracy: 0.25    |  0:00:01s
epoch 16 | loss: 0.39329

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-26 07:51:29,788] Trial 0 finished with value: 0.5 and parameters: {'n_d': 19, 'n_a': 30, 'n_steps': 5, 'gamma': 1.556564525825605, 'lambda_sparse': 3.1173888195380574e-05, 'learning_rate': 0.002232208624417936, 'batch_size': 256, 'num_epochs': 96}. Best is trial 0 with value: 0.5.


...

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-26 07:54:03,463] Trial 49 finished with value: 0.75 and parameters: {'n_d': 23, 'n_a': 29, 'n_steps': 9, 'gamma': 1.1641557272828789, 'lambda_sparse': 6.600465112439704e-05, 'learning_rate': 0.007821917938746453, 'batch_size': 32, 'num_epochs': 55}. Best is trial 7 with value: 1.0.


epoch 13 | loss: 0.29652 | val_accuracy: 0.625   |  0:00:02s

Early stopping occurred at epoch 13 with best_epoch = 3 and best_val_accuracy = 0.75


c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.67921 | val_accuracy: 0.0     |  0:00:00s
epoch 1  | loss: 2.58568 | val_accuracy: 0.25    |  0:00:00s
epoch 2  | loss: 1.93807 | val_accuracy: 0.25    |  0:00:00s
epoch 3  | loss: 1.46323 | val_accuracy: 0.5     |  0:00:01s
epoch 4  | loss: 0.95907 | val_accuracy: 0.5     |  0:00:01s
epoch 5  | loss: 1.46162 | val_accuracy: 0.625   |  0:00:01s
epoch 6  | loss: 1.0283  | val_accuracy: 0.875   |  0:00:01s
epoch 7  | loss: 0.95073 | val_accuracy: 0.875   |  0:00:02s
epoch 8  | loss: 0.90242 | val_accuracy: 0.875   |  0:00:02s
epoch 9  | loss: 0.89346 | val_accuracy: 0.875   |  0:00:02s
epoch 10 | loss: 1.30834 | val_accuracy: 0.875   |  0:00:02s
epoch 11 | loss: 0.55264 | val_accuracy: 0.875   |  0:00:02s
epoch 12 | loss: 0.68679 | val_accuracy: 1.0     |  0:00:02s
epoch 13 | loss: 0.79067 | val_accuracy: 1.0     |  0:00:02s
epoch 14 | loss: 0.68402 | val_accuracy: 0.75    |  0:00:03s
epoch 15 | loss: 0.37625 | val_accuracy: 0.75    |  0:00:03s
epoch 16 | loss: 0.74194

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [63]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class KAN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(KAN, self).__init__()
        self.hidden_layer = nn.Linear(input_dim, hidden_dim)
        self.activation = nn.ReLU()
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h = self.activation(self.hidden_layer(x))
        out = self.output_layer(h)
        return out

def objective(trial):
    # Define hyperparameters to tune for KAN
    hidden_dim = trial.suggest_int('hidden_dim', 32, 256)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the KAN model
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    model = KAN(input_dim, hidden_dim, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final KAN model with the best hyperparameters
input_dim = X_train.shape[1]
output_dim = len(np.unique(y))
best_model = KAN(input_dim, 
                 best_params['hidden_dim'], 
                 output_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_tensor)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    
    _, predicted = torch.max(outputs, 1)
    y_pred = predicted.cpu().numpy()
    proba = torch.softmax(outputs, dim=1).cpu().numpy()

    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        auc = roc_auc_score(y_true, proba[:, 1])
    else:  # Multi-class classification
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['KAN'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 07:54:08,233] A new study created in memory with name: no-name-31e1a594-edde-41c5-8805-881e363c58f3


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\560273739.py:53: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:54:08,703] Trial 0 finished with value: 0.75 and parameters: {'hidden_dim': 32, 'learning_rate': 0.008869898978533712, 'batch_size': 128, 'num_epochs': 83}. Best is trial 0 with value: 0.75.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\560273739.py:53: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 07:54:08,881] Trial 1 finished with value: 1.0 and parameters: {'hidden_dim': 86, 'learning_

                           Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression            0.75  0.927778                    0.002991   
KNN                            0.75       1.0                    0.000996   
Decision Tree                 0.875  0.931746                       0.001   
Random Forest                  0.75  0.933333                    0.118717   
Gradient Boosting             0.875  0.920635                    0.247005   
XGBoost                        0.75  0.805556                    0.082778   
LightGBM                      0.125       0.5                    0.010972   
CatBoost                       0.75  0.977778                    0.119442   
MLP                            0.75       1.0                    0.513627   
DNN                            0.75  0.922222                     0.23946   
DCN                           0.875  0.927778                    0.297203   
Wide_and_Deep                 0.875  0.830556                    0.449732   

In [64]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class SAINT(nn.Module):
    def __init__(self, input_dim, num_classes, dim, depth, heads, mlp_dim, dropout=0.1):
        super(SAINT, self).__init__()
        self.embeds = nn.Linear(input_dim, dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim, dropout=dropout),
            num_layers=depth
        )
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, x):
        x = self.embeds(x)
        x = x.unsqueeze(1)  # Add sequence dimension
        x = self.transformer(x)
        x = x.squeeze(1)  # Remove sequence dimension
        return self.mlp_head(x)

def objective(trial):
    # Define hyperparameters to tune for SAINT
    heads = trial.suggest_int('heads', 1, 8)
    dim = trial.suggest_int('dim', heads, 256, step=heads)  # Ensure dim is divisible by heads
    depth = trial.suggest_int('depth', 1, 6)
    mlp_dim = trial.suggest_int('mlp_dim', 32, 256)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the SAINT model
    input_dim = X_train.shape[1]
    num_classes = len(np.unique(y))
    model = SAINT(input_dim, num_classes, dim, depth, heads, mlp_dim, dropout).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final SAINT model with the best hyperparameters
input_dim = X_train.shape[1]
num_classes = len(np.unique(y))
best_model = SAINT(input_dim, num_classes, 
                   best_params['dim'], 
                   best_params['depth'], 
                   best_params['heads'], 
                   best_params['mlp_dim'], 
                   best_params['dropout']).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_tensor)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    
    _, predicted = torch.max(outputs, 1)
    y_pred = predicted.cpu().numpy()
    proba = torch.softmax(outputs, dim=1).cpu().numpy()

    accuracy = accuracy_score(y_true, y_pred)
    
    if num_classes == 2:  # Binary classification
        auc = roc_auc_score(y_true, proba[:, 1])
    else:  # Multi-class classification
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['SAINT'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 07:54:25,227] A new study created in memory with name: no-name-d918826b-935d-4448-ba2c-793621b5c467


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\1518152593.py:65: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
[I 2024-07-26 07:54:28,455] Trial 0 finished with value: 0.5 and parameters: {'heads': 1, 'dim': 3, 'depth': 3, 'mlp_dim': 107, 'dropout': 0.27778102782570907, 'learning_rate': 0.0001716030028123978, 'batch_size': 32, 'num_epochs': 32}. Best is trial 0 with value: 0.

                           Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression            0.75  0.927778                    0.002991   
KNN                            0.75       1.0                    0.000996   
Decision Tree                 0.875  0.931746                       0.001   
Random Forest                  0.75  0.933333                    0.118717   
Gradient Boosting             0.875  0.920635                    0.247005   
XGBoost                        0.75  0.805556                    0.082778   
LightGBM                      0.125       0.5                    0.010972   
CatBoost                       0.75  0.977778                    0.119442   
MLP                            0.75       1.0                    0.513627   
DNN                            0.75  0.922222                     0.23946   
DCN                           0.875  0.927778                    0.297203   
Wide_and_Deep                 0.875  0.830556                    0.449732   

In [65]:
import pandas as pd
import numpy as np
import time
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

def objective(trial):
    # Define hyperparameters to tune for VIME-like model (using TabNet as proxy)
    n_d = trial.suggest_int('n_d', 8, 64)
    n_a = trial.suggest_int('n_a', 8, 64)
    n_steps = trial.suggest_int('n_steps', 3, 10)
    gamma = trial.suggest_float('gamma', 1.0, 2.0)
    lambda_sparse = trial.suggest_loguniform('lambda_sparse', 1e-6, 1e-3)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the TabNet model
    model = TabNetClassifier(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        lambda_sparse=lambda_sparse,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=learning_rate),
        device_name=device
    )

    # Training
    model.fit(
        X_train=X_train_scaled, y_train=y_train.values,
        eval_set=[(X_test_scaled, y_test.values)],
        eval_name=['val'],
        eval_metric=['accuracy'],
        max_epochs=num_epochs,
        patience=10,
        batch_size=batch_size,
        virtual_batch_size=batch_size // 2,
        num_workers=0,
        drop_last=False
    )

    # Evaluation
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final TabNet model with the best hyperparameters
best_model = TabNetClassifier(
    n_d=best_params['n_d'],
    n_a=best_params['n_a'],
    n_steps=best_params['n_steps'],
    gamma=best_params['gamma'],
    lambda_sparse=best_params['lambda_sparse'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=best_params['learning_rate']),
    device_name=device
)

training_start_time = time.time()
best_model.fit(
    X_train=X_train_scaled, y_train=y_train.values,
    eval_set=[(X_test_scaled, y_test.values)],
    eval_name=['val'],
    eval_metric=['accuracy'],
    max_epochs=best_params['num_epochs'],
    patience=10,
    batch_size=best_params['batch_size'],
    virtual_batch_size=best_params['batch_size'] // 2,
    num_workers=0,
    drop_last=False
)
training_time = time.time() - training_start_time

# Evaluation
y_pred = best_model.predict(X_test_scaled)
inference_start_time = time.time()
y_pred_proba = best_model.predict_proba(X_test_scaled)
inference_time = time.time() - inference_start_time

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
if len(np.unique(y)) == 2:  # Binary classification
    auc = roc_auc_score(y_test, y_pred_proba[:, 1])
else:  # Multiclass classification
    auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['VIME'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 07:57:32,222] A new study created in memory with name: no-name-9d43ab35-eb79-4e26-a516-277b27d33e12


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\2451662473.py:43: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lambda_sparse = trial.suggest_loguniform('lambda_sparse', 1e-6, 1e-3)
C:\Users\Tommy\AppData\Local\Temp\ipykernel_8880\2451662473.py:44: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.50374 | val_accuracy: 0.375   |  0:00:00s
epoch 1  | loss: 1.89285 | val_accuracy: 0.375   |  0:00:00s
epoch 2  | loss: 1.9433  | val_accuracy: 0.75    |  0:00:00s
epoch 3  | loss: 1.58311 | val_accuracy: 0.625   |  0:00:00s
epoch 4  | loss: 1.61411 | val_accuracy: 0.5     |  0:00:00s
epoch 5  | loss: 1.3291  | val_accuracy: 0.625   |  0:00:00s
epoch 6  | loss: 1.40681 | val_accuracy: 0.5     |  0:00:00s
epoch 7  | loss: 1.21003 | val_accuracy: 0.625   |  0:00:01s
epoch 8  | loss: 1.38053 | val_accuracy: 0.75    |  0:00:01s
epoch 9  | loss: 1.34901 | val_accuracy: 0.875   |  0:00:01s
epoch 10 | loss: 1.09436 | val_accuracy: 0.875   |  0:00:01s
epoch 11 | loss: 1.05671 | val_accuracy: 0.75    |  0:00:01s
epoch 12 | loss: 1.0244  | val_accuracy: 0.625   |  0:00:01s
epoch 13 | loss: 0.77091 | val_accuracy: 0.625   |  0:00:01s
epoch 14 | loss: 0.85441 | val_accuracy: 0.625   |  0:00:02s
epoch 15 | loss: 0.63634 | val_accuracy: 0.5     |  0:00:02s
epoch 16 | loss: 0.65374

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-26 07:57:35,108] Trial 0 finished with value: 0.875 and parameters: {'n_d': 11, 'n_a': 16, 'n_steps': 10, 'gamma': 1.6273843140950668, 'lambda_sparse': 6.757651400226338e-05, 'learning_rate': 0.005768493906323893, 'batch_size': 64, 'num_epochs': 35}. Best is trial 0 with value: 0.875.


...

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-26 07:59:27,141] Trial 49 finished with value: 0.75 and parameters: {'n_d': 60, 'n_a': 29, 'n_steps': 9, 'gamma': 1.915282786924466, 'lambda_sparse': 0.0003325181149945809, 'learning_rate': 0.012373116095869423, 'batch_size': 64, 'num_epochs': 31}. Best is trial 31 with value: 1.0.


epoch 14 | loss: 0.1169  | val_accuracy: 0.625   |  0:00:01s

Early stopping occurred at epoch 14 with best_epoch = 4 and best_val_accuracy = 0.75


c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 3.94984 | val_accuracy: 0.125   |  0:00:00s
epoch 1  | loss: 2.64501 | val_accuracy: 0.25    |  0:00:00s
epoch 2  | loss: 1.8386  | val_accuracy: 0.375   |  0:00:00s
epoch 3  | loss: 1.26554 | val_accuracy: 0.375   |  0:00:00s
epoch 4  | loss: 0.85521 | val_accuracy: 0.5     |  0:00:00s
epoch 5  | loss: 0.54311 | val_accuracy: 0.5     |  0:00:00s
epoch 6  | loss: 0.5092  | val_accuracy: 0.5     |  0:00:00s
epoch 7  | loss: 0.33521 | val_accuracy: 0.5     |  0:00:01s
epoch 8  | loss: 0.40992 | val_accuracy: 0.375   |  0:00:01s
epoch 9  | loss: 0.23863 | val_accuracy: 0.375   |  0:00:01s
epoch 10 | loss: 0.17465 | val_accuracy: 0.5     |  0:00:01s
epoch 11 | loss: 0.31905 | val_accuracy: 0.375   |  0:00:01s
epoch 12 | loss: 0.43888 | val_accuracy: 0.625   |  0:00:01s
epoch 13 | loss: 0.21215 | val_accuracy: 0.625   |  0:00:01s
epoch 14 | loss: 0.23946 | val_accuracy: 0.625   |  0:00:01s
epoch 15 | loss: 0.10086 | val_accuracy: 0.75    |  0:00:01s
epoch 16 | loss: 0.05508

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
